# Space Charge Example

Beyond the <a href="Analyze_MCTruth.ipynb">Analyze_MCTruth</a> notebook example, there's not a whole lot more to show that isn't really more a demonstration of gallery/art-ROOT files and other analysis examples. Let your creativity fly!

But, OK, let's do something a little concrete to show how to get more complicated things going. Suppose we wanted to shift "truth" particle locations to what their apparent position would be after taking into account the space charge effect. This is important if you, for instance, want to compare truth particle locations to reconstructed locations for resolution studies.

I'm currently working with MicroBooNE code/setup here, so I'm going to stick with that. Many experiments have a specific SpaceCharge instance that they use in their simulation and reconstruction. In larsoft, this is called up in an `art::Service`. We don't have Services in gallery (it's outside gallery's scope!), but underneath the service is a more generice `SpaceCharge` class. We can call that up here!

So, let's talk strategy. The `spacecharge::SpaceChargeMicroBooNE` object is constructed with a `fhicl::ParameterSet` that looks like the following:
```
sce_ps:
{
  EnableCorrSCE: false
  EnableSimSpatialSCE: true
  EnableSimEfieldSCE: true 
  RepresentationType: Parametric
  InputFilename:"SpaceCharge/SCEoffsets_MicroBooNE_E273.root
}
```
So, we're going to need a header file that includes the `SpaceChargeMicroBooNE` but we're also going to need a utility to create the ParameterSet input. Luckily, there's a `make_ParameterSet` function from fhicl that will fill in a ParameterSet from a string. Our necessary includes are then:

```
#include "uboone/SpaceCharge/SpaceChargeMicroBooNE.h"
#include "fhiclcpp/ParameterSet.h"
#include "fhiclcpp/make_ParameterSet.h"
```
which I've put in a file: <a href="common/spacecharge_includes.h">common/spacecharge_includes.h</a>. We then load that up like we did before:

In [1]:
gROOT->ProcessLine(".L common/spacecharge_includes.h+O");

OK, so now, let's make our fhicl configuration:

In [2]:
fhicl::ParameterSet pset;
std::string config("EnableCorrSCE:false EnableSimSpatialSCE:true EnableSimEfieldSCE:true RepresentationType:Parametric InputFilename:\"SpaceCharge/SCEoffsets_MicroBooNE_E273.root\"");

In [3]:
//make parameter set
fhicl::make_ParameterSet(config,pset);

//test it to make sure it looks ok:
std::cout << pset.to_string() << std::endl;

EnableCorrSCE:false EnableSimEfieldSCE:true EnableSimSpatialSCE:true InputFilename:"SpaceCharge/SCEoffsets_MicroBooNE_E273.root" RepresentationType:"Parametric"


Now we can make a spacecharge object. BUT, if you execute this in the next cell:
```
spacecharge::SpaceChargeMicroBooNE sce(pset);
```
you will get an error message like:
```
IncrementalExecutor::executeFunction: symbol '_ZN11spacecharge21SpaceChargeMicroBooNEC1ERKN5fhicl12ParameterSetE' unresolved while linking function '_GLOBAL__sub_I_cling_module_102'!
You are probably missing the definition of spacecharge::SpaceChargeMicroBooNE::SpaceChargeMicroBooNE(fhicl::ParameterSet const&)
Maybe you need to load the corresponding shared library?
IncrementalExecutor::executeFunction: symbol 'gsl_interp_free' unresolved while linking function '_GLOBAL__sub_I_cling_module_102'!
IncrementalExecutor::executeFunction: symbol '_ZTVN11spacecharge21SpaceChargeMicroBooNEE' unresolved while linking function '_GLOBAL__sub_I_cling_module_102'!
You are probably missing the definition of vtable for spacecharge::SpaceChargeMicroBooNE
Maybe you need to load the corresponding shared library?
```
It's a linking error: you'd get a similar thing from a compiler. Note the symbols missing: gsl_interp and a vtable definition for SpaceChargeMicroBooNE. Now, those libraries exist, we just need to tell ROOT to link against them. So, we have to more ProcessLine commands to do:

In [4]:
gROOT->ProcessLine(".L $GSL_LIB/libgsl.so");
gROOT->ProcessLine(".L $UBOONECODE_LIB/libuboone_SpaceCharge.so");

And, ok, _now_ we can setup the space charge object.

In [5]:
spacecharge::SpaceChargeMicroBooNE sce(pset);

Alright, let's test it out a little. If we give a `geo::Point_t` for a position in the TPC, we should get back the spatial distortion vector.

In [6]:
//setup our inputs and outputs
geo::Point_t pt,pt_offset;
geo::Vector_t diff_vec;

In [7]:
pt = {200.0,100.0,100.0};
diff_vec = sce.GetPosOffsets(pt);
pt_offset = pt+diff_vec;
std::cout << "Position offsets: (" << diff_vec.X() << " " << diff_vec.Y() << " " << diff_vec.Z() << ")" << std::endl;
std::cout << "(" << pt.X() << " " << pt.Y() << " " << pt.Z() << ") --> "
          << "(" << pt_offset.X() << " " << pt_offset.Y() << " " << pt_offset.Z() << ")" << std::endl;

Position offsets: (-1.35893 -8.99825 0.784353)
(200 100 100) --> (198.641 91.0018 100.784)


OK, that's nice. But come on, we can do better. Let's make a 3D function that we can visualize!

It's usually best practive when declaring functions that will be used later to use the `%%cpp -d` magic.

In [8]:
%%cpp -d
Double_t SpaceChargeMagnitude(Double_t *x, Double_t *par)
{
    return (sce.GetPosOffsets({x[0],x[1],x[2]})).R();
}
Double_t SpaceChargeMagnitudeX(Double_t *x, Double_t *p)
{
    return (sce.GetPosOffsets({p[0],x[0],x[1]})).R();
}
Double_t SpaceChargeMagnitudeY(Double_t *x, Double_t *p)
{
    return (sce.GetPosOffsets({x[0],p[0],x[1]})).R();
}
Double_t SpaceChargeMagnitudeZ(Double_t *x, Double_t *p)
{
    return (sce.GetPosOffsets({x[0],x[1],p[0]})).R();
}

In [9]:
TF3 fSCE_mag("fSCE_mag",SpaceChargeMagnitude,0.0,256.,-116.5,116.5,0.0,1037.0);
TF2 fSCE_mag_xproj("fSCE_mag_xproj",SpaceChargeMagnitudeX,-116.5,116.5,0.0,1037.0,1);
TF2 fSCE_mag_yproj("fSCE_mag_yproj",SpaceChargeMagnitudeY,0.0,256.,0.0,1037.0,1);
TF2 fSCE_mag_zproj("fSCE_mag_zproj",SpaceChargeMagnitudeZ,0.0,256.,-116.5,116.5,1);

In [10]:
TCanvas c("c");

In [11]:
fSCE_mag.Eval(200.0,100.,100.)

(double) 9.1340209


In [12]:
%jsroot on
c.cd();
fSCE_mag_zproj.Draw("colz");
c.Draw();